In [1]:
from pyarrow import parquet
import pyarrow
import datasets

import transformers
import numpy as np

from tqdm.notebook import tqdm


DATASET = "../../dataset_processed_discernia/prm800k.train.parquet"
TOKENIZER_NAME = "imone/LLaMA2_13B_with_correctness_token"

# Load tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(TOKENIZER_NAME)

In [3]:
ds_parquet = parquet.read_table(DATASET)

In [6]:
supervise_len = np.array([pyarrow.compute.sum(x.values).as_py() for x in tqdm(ds_parquet.column("0_masks"))])

  0%|          | 0/94470 [00:00<?, ?it/s]

In [8]:
# Length statistics
lengths = np.array(ds_parquet.column("total_length").to_pylist())

In [10]:
np.mean(lengths)

324.22891923361914

In [14]:
# Print the supervised and unsupervised text
sample_index = 1001

item = ds_parquet.take([sample_index]).to_pylist()[0]

token_ids = np.array(item["0_tokens"])
masks     = np.array(item["0_masks"])

print(tokenizer.decode(token_ids[~masks].tolist(), spaces_between_special_tokens=False).replace("<|end_of_turn|>", "<|end_of_turn|>\n\n"))
print("=================")
print(tokenizer.decode(token_ids[masks].tolist(), spaces_between_special_tokens=False).replace("<|end_of_turn|>", "<|end_of_turn|>\n\n"))

<s> Provide a wrong solution to the user's problem.<|end_of_turn|>

 User: Let $a_n$ be the integer obtained by writing all the integers from $1$ to $n$ from left to right. For example, $a_3 = 123$ and $a_{11} = 1234567891011$. Compute the remainder when $a_{44}$ is divided by $45$.<|end_of_turn|>


<|correct|> This is an interesting problem that involves some number theory and modular arithmetic.<|end_of_step|><|correct|> I notice that 45 is divisible by 9, so I can use the fact that a number is divisible by 9 if and only if the sum of its digits is divisible by 9.<|end_of_step|><|wrong|> This means that the remainder when $a_{44}$ is divided by 45 is the same as the remainder when the sum of the digits of $a_{44}$ is divided^{9}{45} .<|end_of_step|><|end_of_turn|>


